# Developing a Chatbot with Python using Machine Learning

As we are developing a Chatbot with Python using Machine Learning, we need some data to train our model.We can just create our own dataset to train the model.

To create this dataset to create a chatbot with Python, we need to understand what intents we are going to train. An “intention” is the user’s intention to interact with a chatbot or the intention behind every message the chatbot receives from a particular user.

**The strategy here is to set different intents and create training samples for those intents and train your chatbot model with these sample training data as model training data (X) and intents in as model training categories (Y).**

# **Data preparation:**


The second step of this task to create a chatbot with Python and Machine Learning is to prepare the data to train our chatbot. I’ll start this step by importing the necessary libraries and packages:

In [1]:
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

# Read the JSON file and process the required files:

In [2]:
with open('intents.json') as file:
   data = json.load(file)

training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])

    if intent['tag'] not in labels:
        labels.append(intent['tag'])

num_classes = len(labels)

# Use the label encoder method provided by the Scikit-Learn library in Python:

In [3]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

# Tokenization:

Now we need to vectorize the data using the Tokenization method to create a chatbot with Python and Machine Learning:

In [4]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

# Training a Neural Network

In [5]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

model.summary()
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (  (None, 16)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 8)                 136       
                                                                 
Total params: 16680 (65.16 KB)
Trainable params: 16680 (65.16 KB)
Non-trainable params: 0 (0.00 Byte)
____________________

let’s save the model so that we can use this neural network in the future as well:

In [6]:
# to save the trained model
model.save("chat_model")

import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

# Now let’s Build a Chatbot with Python and our Trained Machine Learning Model

Now we are implementing a chat function to interact with a real user. When the message from the user will be received, the chatbot will compute the similarity between the sequence of the new text and the training data.


Taking into account the trust scores obtained for each category, it categorizes the user’s message according to an intention with the highest trust score:

In [9]:
import json
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder
import colorama
import random
import pickle

# Install colorama if not already installed
!pip install colorama

colorama.init()
from colorama import Fore, Style

# Load data from intents.json
with open("intents.json") as file:
    data = json.load(file)

def load_model_and_tokenizer():
    """Load the trained model, tokenizer, and label encoder."""
    model = keras.models.load_model('chat_model')
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)
    return model, tokenizer, lbl_encoder

def get_response(model, tokenizer, lbl_encoder, input_text, max_len=20):
    """Generate a response based on the input text."""
    result = model.predict(keras.preprocessing.sequence.pad_sequences(
        tokenizer.texts_to_sequences([input_text]),
        truncating='post', maxlen=max_len))
    tag = lbl_encoder.inverse_transform([np.argmax(result)])

    for intent in data['intents']:
        if intent['tag'] == tag:
            return random.choice(intent['responses'])

def chat():
    """Start the chatbot conversation."""
    model, tokenizer, lbl_encoder = load_model_and_tokenizer()

    print(Fore.YELLOW + "Start messaging with the bot (type 'quit' to stop)!" + Style.RESET_ALL)

    while True:
        user_input = input(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL)
        if user_input.lower() == "quit":
            break
        response = get_response(model, tokenizer, lbl_encoder, user_input)
        if response:
            print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL, response)

if __name__ == "__main__":
    chat()


Start messaging with the bot (type 'quit' to stop)!
User: hello
1/1 [==============================] - 0s 89ms/step
ChatBot: Hi there
User: who are you
1/1 [==============================] - 0s 23ms/step
ChatBot: I'm Joana, an Artificial Intelligent bot
User: bye
1/1 [==============================] - 0s 25ms/step
ChatBot: My pleasure


KeyboardInterrupt: Interrupted by user